# How to Deploy and Interact with Solidity Contracts with Python and Ganache

In [1]:
# Installing the Package
#!pip install web3
#!pip install py-solc-x
#!pip install python-dotenv

In [2]:
# Importing Necessary the Package
from web3 import Web3
import os
from solcx import compile_standard, install_solc
from dotenv import load_dotenv
import json
load_dotenv()

True

Now that the environment is setup, what exactly do you do to deploy a contract?

In [3]:
# Read the bank.sol file in Python
with open("../contract/bank.sol", "r") as file:
    solidity_contact = file.read()

In [4]:
solidity_contact

'pragma solidity 0.6.0;\n\ncontract Bank\n{\n    int bal;\n\n    constructor() public\n    {\n        bal = 1;\n    }\n\n    function getBalance() view public returns(int)\n    {\n        return bal;\n    }\n\n    function withdraw(int amt) public\n    {\n        bal = bal - amt;\n    }\n\n    function deposit(int amt) public\n    {\n        bal = bal + amt;\n    }\n}\n\n\n'

Now, let me go back one step back. The modules you need from solcx are compile_standard and install_solc — the first to compile and the second to invoke a solc version which you want to use.

In [5]:
install_solc("0.6.0")

Version('0.6.0')

In [6]:
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"Bank.sol": {"content": solidity_contact}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.6.0",
)
compiled_sol

{'contracts': {'Bank.sol': {'Bank': {'abi': [{'inputs': [],
      'stateMutability': 'nonpayable',
      'type': 'constructor'},
     {'inputs': [{'internalType': 'int256', 'name': 'amt', 'type': 'int256'}],
      'name': 'deposit',
      'outputs': [],
      'stateMutability': 'nonpayable',
      'type': 'function'},
     {'inputs': [],
      'name': 'getBalance',
      'outputs': [{'internalType': 'int256', 'name': '', 'type': 'int256'}],
      'stateMutability': 'view',
      'type': 'function'},
     {'inputs': [{'internalType': 'int256', 'name': 'amt', 'type': 'int256'}],
      'name': 'withdraw',
      'outputs': [],
      'stateMutability': 'nonpayable',
      'type': 'function'}],
    'evm': {'bytecode': {'linkReferences': {},
      'object': '608060405234801561001057600080fd5b50600160008190555061010e806100286000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c806312065fe01460415780637e62eab814605d578063f04991f0146088575b600080fd5b604760b3565b60405180828152

In [7]:
with open("../compile/bank.json", "w") as file:
    json.dump(compiled_sol, file)    

**Compiling Solidity Smart Contracts in Python**

compile_standard has got a json based layout where you tell the compiler what do you want to do and what to expect as output. What are the inputs

1. Language
2. Source File and how you invoke the source file(in our case, the python variable created)
3. Output: in this case, ABI, Metadata, Bytecode and SourceMap.
4. Solidity Version to use.


In [8]:
# get bytecode
bytecode = compiled_sol["contracts"]["Bank.sol"]["Bank"]["evm"]["bytecode"]["object"]
bytecode

'608060405234801561001057600080fd5b50600160008190555061010e806100286000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c806312065fe01460415780637e62eab814605d578063f04991f0146088575b600080fd5b604760b3565b6040518082815260200191505060405180910390f35b608660048036036020811015607157600080fd5b810190808035906020019092919050505060bc565b005b60b160048036036020811015609c57600080fd5b810190808035906020019092919050505060ca565b005b60008054905090565b806000540360008190555050565b80600054016000819055505056fea2646970667358221220ac2f9ac0414ca7b2604e93653aca76b900f848f86af45bde45b864c1b778628064736f6c63430006000033'

In [9]:
# get abi (Application Binary Interface)
abi = json.loads(
    compiled_sol["contracts"]["Bank.sol"]["Bank"]["metadata"])["output"]["abi"]
abi

[{'inputs': [], 'stateMutability': 'nonpayable', 'type': 'constructor'},
 {'inputs': [{'internalType': 'int256', 'name': 'amt', 'type': 'int256'}],
  'name': 'deposit',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [],
  'name': 'getBalance',
  'outputs': [{'internalType': 'int256', 'name': '', 'type': 'int256'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'internalType': 'int256', 'name': 'amt', 'type': 'int256'}],
  'name': 'withdraw',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'}]

**Extracting the ABI and Bytecode from Solidity Smart Contracts in Python**

ABI or Application Binary Interface which holds the structure of your contract.

There are two things to notice here — name and stateMutability. Name is the name of the object and stateMutability is the kind of object it is — whether it’s a viewable object or an executable function.

![deploy1.png](../static/image/deploy/deploy1.png)

![deploy2.png](../static/image/deploy/deploy2.png)


If you look into Remix IDE, you will see this difference clearly. The orange one is a modifiable object(stateMutability = nonpayable) and the blue ones are view variables.

Now, fetch the values for byteCode and abi. You just need to walk through the structure.

## What is Ganache?
Ganache is a platform to perform blockchain operations locally on our machine before deploying a not-perfect file or how we want it to be. In other words, it’s a test blockchain where you can track transactions, inspect changes, and, most importantly– do it locally by creating your own Ethereum blockchain.

Depending on whether you like a UI or CLI, you can use Ganache CLI or Ganache.

**Connect using Ganache- Local Ethereum Blockchain to deploy Smart Contracts**

For deploying the code, you need a few things —

1. Destination server(RPC Server)
2. Chain ID: This is used in transaction signature process
3. Network ID: This is needed for P2P Communication between nodes.
4. Address and Private Key

Generally, Chain ID and Network ID are kept same for testing



In [10]:
# set up connection
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))
w3

In [11]:
chain_id = int(os.getenv('chain_id'))
#address = os.getenv('my_address')
private_key = os.getenv("private_key")
w3.eth.defaultAccount = w3.eth.accounts[0]
address = w3.eth.defaultAccount
address

'0xfe5Bb56F15de2251ed7fd6EE7E2d7D95BCc54DB6'

With all the information available, it’s time to deploy the contract. The steps —

1. Convert the compiled code into an ethereum contract object
2. Convert the object into a transaction. Nonce will be the nonce of the server.
3. Sign the transaction
4. Send the transaction to the blockchain
5. Verify the receipt.

In [12]:
# convert this into a ethereum contract object.
ContactList = w3.eth.contract(abi=abi, bytecode=bytecode)
ContactList

web3._utils.datatypes.Contract

In [13]:
# Get the number of latest transaction
nonce = w3.eth.getTransactionCount(address)
nonce

3

In [14]:
# Build Transaction
transaction = ContactList.constructor().buildTransaction(
                {
                    "chainId": chain_id,
                    "gasPrice": w3.eth.gas_price,
                    "from": address,
                    "nonce": nonce,
                }
            )
transaction

{'value': 0,
 'gas': 131815,
 'chainId': 1337,
 'gasPrice': 20000000000,
 'from': '0xfe5Bb56F15de2251ed7fd6EE7E2d7D95BCc54DB6',
 'nonce': 3,
 'data': '0x608060405234801561001057600080fd5b50600160008190555061010e806100286000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c806312065fe01460415780637e62eab814605d578063f04991f0146088575b600080fd5b604760b3565b6040518082815260200191505060405180910390f35b608660048036036020811015607157600080fd5b810190808035906020019092919050505060bc565b005b60b160048036036020811015609c57600080fd5b810190808035906020019092919050505060ca565b005b60008054905090565b806000540360008190555050565b80600054016000819055505056fea2646970667358221220ac2f9ac0414ca7b2604e93653aca76b900f848f86af45bde45b864c1b778628064736f6c63430006000033',
 'to': b''}

In [15]:
# Sign the transaction
sign_transaction = w3.eth.account.signTransaction(transaction, private_key=private_key)
sign_transaction

SignedTransaction(rawTransaction=HexBytes('0xf9018b038504a817c800830202e78080b90136608060405234801561001057600080fd5b50600160008190555061010e806100286000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c806312065fe01460415780637e62eab814605d578063f04991f0146088575b600080fd5b604760b3565b6040518082815260200191505060405180910390f35b608660048036036020811015607157600080fd5b810190808035906020019092919050505060bc565b005b60b160048036036020811015609c57600080fd5b810190808035906020019092919050505060ca565b005b60008054905090565b806000540360008190555050565b80600054016000819055505056fea2646970667358221220ac2f9ac0414ca7b2604e93653aca76b900f848f86af45bde45b864c1b778628064736f6c63430006000033820a96a065c9f5a91b563aed74f08f5e1a6b6865498f6378cc4b0be6b0dea6c4ae4f5169a06f9675289d370ed2c4ea264a44abcc0ceed25021743ed5e2e4b4077137f31146'), hash=HexBytes('0x8787765005537f1e5b75061346a7f4c0b1369e81a85dd360835fe4afa65e1373'), r=46040429453895215828518510866743476798946783988876601873671140420893

In [16]:
# Send the transaction
transaction_hash = w3.eth.sendRawTransaction(sign_transaction.rawTransaction)
transaction_hash

HexBytes('0x8787765005537f1e5b75061346a7f4c0b1369e81a85dd360835fe4afa65e1373')

In [17]:
# Wait for the transaction to be mined, and get the transaction receipt
transaction_receipt = w3.eth.wait_for_transaction_receipt(transaction_hash)
transaction_receipt

AttributeDict({'transactionHash': HexBytes('0x8787765005537f1e5b75061346a7f4c0b1369e81a85dd360835fe4afa65e1373'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x6b1aff307ae64c70cd7731b941978ff6bdac421625e698d47ba38864928b4d6b'),
 'blockNumber': 4,
 'from': '0xfe5Bb56F15de2251ed7fd6EE7E2d7D95BCc54DB6',
 'to': None,
 'gasUsed': 131815,
 'cumulativeGasUsed': 131815,
 'contractAddress': '0x78e4993370572aAFCD29204E1cC9D40C050B740F',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

If you want to interact with a blockchain, you will do that with either a call or a transact. Call is used to read data from the blockchain, and transact is used to write data to the blockchain. Call function is simple. Fetch the object from ABI and invoke it.

In [18]:
contract_list = w3.eth.contract(address=transaction_receipt.contractAddress, abi=abi)
contract_list

Transact is exactly identical to deploying a concept — but with the specific function you want to invoke.

In [19]:
def currentBalance():
    return contract_list.functions.getBalance().call()


def depositMoney(amt):
    contract_list.functions.deposit(amt).transact({'from': address})


def withdrawMoney(amt):
    contract_list.functions.withdraw(amt).transact({'from': address})

In [20]:
currentBalance()

1

In [21]:
depositMoney(100)

In [22]:
currentBalance()


101

In [23]:
withdrawMoney(10)

In [24]:
currentBalance()

91